In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

2023-12-04 12:15:57.967860: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


2023-12-04 12:17:36.690488: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-12-04 12:17:36.779605: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-12-04 12:17:36.779971: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [3]:
# Root label (source = ASRS coding forms) : order = by descending frequency
anomaly_labels=['Deviation / Discrepancy - Procedural',
                    'Aircraft Equipment',
                    'Conflict',
                    'Inflight Event / Encounter',
                    'ATC Issue',
                    'Deviation - Altitude',
                    'Deviation - Track / Heading',
                    'Ground Event / Encounter',
                    'Flight Deck / Cabin / Aircraft Event',
                    'Ground Incursion',
                    'Airspace Violation',
                    'Deviation - Speed',
                    'Ground Excursion',
                    'No Specific Anomaly Occurred']

In [4]:
def load_data(path, labels, add_other=False):
    loaded_data = pd.read_pickle(path)[0]

    # Drop Anomaly NaN's
    loaded_data = loaded_data.dropna(subset=['Anomaly']).reset_index(drop=True)

    # Convert the 'Anomaly' column to a list of lists
    anomaly_series = loaded_data['Anomaly']
    anomaly_list = anomaly_series.str.split(';').apply(lambda x: [item.strip() for item in x])

    # Initialize a DataFrame to hold the one-hot-encoded anomalies
    anomaly_df = pd.DataFrame(index=loaded_data.index)

    # Populate the DataFrame with one-hot-encoded columns for each prefix
    for prefix in labels:
        anomaly_df[prefix] = anomaly_list.apply(lambda anomalies: any(anomaly.startswith(prefix) for anomaly in anomalies)).astype(int)

    # Add the 'Other' category
    if add_other:
        anomaly_df['Other'] = (anomaly_df.sum(axis=1) == 0).astype(int)

    # Assign the one-hot-encoded anomalies as a new column 'labels' to 'loaded_data'
    loaded_data['labels'] = anomaly_df.apply(lambda row: row.tolist(), axis=1)

    # Now, 'loaded_data' is a DataFrame that includes both the 'text' and 'labels' columns
    loaded_data['text'] = loaded_data["Narrative"]

    # If you want to create a new DataFrame with just 'text' and 'labels':
    final_df = loaded_data[['text', 'labels']]
    return final_df

In [5]:

df_train = load_data("./data/train_data_final.pkl", anomaly_labels)
df_train



,text,labels
0,I was the pilot flying performing the takeoff....,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,We had 6 shipments of dry ice for the flight; ...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,I have seen a lot of mistakes on every flight ...,"[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]"
3,It was my first time flying into KEUG and I wa...,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,I am writing this report to bring attention to...,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
...,...,...
96981,WE WERE ENRTE IN LNAV AT FL310; 30 MI N OF ATL...,"[1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
96982,CLRED BY TWR CTL TO CROSS RWY 8R/26L AT TXWY E...,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]"
96983,WHILE WORKING NUMEROUS CVG AND CMH DEPS AT A C...,"[0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
96984,ON MIDNIGHT SHIFT; APPROX XA00 LCL TIME; 2 SEC...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [6]:
df_test = load_data("./data/test_data_final.pkl", anomaly_labels)
df_test

,text,labels
0,Flying into SLC on the DELTA THREE RNAV arriva...,"[1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]"
1,ORD was on a very busy east flow arrival push....,"[1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,B737-800 was vectored to an ILS Runway 16L app...,"[1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,We were on a 6 mile final when tower cleared a...,"[1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0]"
4,During Climb we Leveled at 17;000 departure sw...,"[1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]"
...,...,...
10800,FO was flying a visual approach to runway 26 i...,"[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
10801,While assembling a GE C2 transfer gearbox; I n...,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
10802,Nearing the end of a hot; bumpy four-hour IFR ...,"[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
10803,On approach gear went down and noticed yellow ...,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [7]:
from datasets import Dataset, DatasetDict
# use from_dict() and not from_pandas(); otherwise you get an extra key, smth litke '__index col__'
train_dataset = Dataset.from_dict(df_train)
# validation_dataset = Dataset.from_dict(df_valid)
test_dataset = Dataset.from_dict(df_test)

# inspect this object
train_dataset['labels'][:3]

[[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]]

In [8]:
from transformers import AutoTokenizer

# Specify the model name for tokenizer
model_name = "bert-base-uncased" # or any other model you are using

# Instantiate tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Define a tokenization function with max_length of 200
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=200)

# Apply tokenization to your datasets
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)

# Convert to TensorFlow datasets if using TensorFlow models
batch_size = 32

# Instantiate a data_collator
from transformers import DefaultDataCollator
data_collator = DefaultDataCollator(return_tensors="tf")

tf_train_dataset = tokenized_train_dataset.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=True,
    collate_fn=data_collator,  # Defined earlier in your code
    batch_size=batch_size
)

tf_test_dataset = tokenized_test_dataset.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=False,
    collate_fn=data_collator,  # Defined earlier in your code
    batch_size=batch_size
)

# Now, you can either train your model using tf_train_dataset or use it for inference.
# For training:
# model.fit(tf_train_dataset, validation_data=tf_validation_dataset, epochs=num_epochs)

# For inference:
# predictions = model.predict(tf_test_dataset)


Map:   0%|          | 0/96986 [00:00<?, ? examples/s]

Map:   0%|          | 0/10805 [00:00<?, ? examples/s]

/home/gonem/miniforge3/envs/ENSAI/lib/python3.8/site-packages/datasets/arrow_dataset.py:400: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(
2023-12-04 12:18:56.529974: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2

In [9]:
# Define the path to your saved model directory
model_directory = 'model_save/7_6_1_UNfrozen_2022_09_14'

# Load the model
loaded_model = tf.keras.models.load_model(model_directory)

In [10]:
loaded_model.evaluate(tf_test_dataset)

2023-12-04 12:20:10.633958: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


338/338 [==============================] - 95s 277ms/step - loss: 0.1720 - binary_accuracy: 0.9311


[0.17195776104927063, 0.9310636520385742]